In [4]:
from IPython.display import Markdown, display
from openai import OpenAI
import json
import os
from time import sleep
import multiprocessing
from multiprocessing.pool import ThreadPool

In [5]:
client = OpenAI()

In [33]:
gpt4o = "gpt-4o-2024-08-06"
gpt4o_mini = "gpt-4o-mini-2024-07-18"
gpt_o1 = "o1-preview"

small_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-small-dataset:AYDdDFb6"
medium_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-medium-dataset:AYE8fROA"
large_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-large-dataset:AYEAuS9Z"
large_dataset_model_ckpt_1046 = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-large-dataset:AYEAuIM0:ckpt-step-1046"

small_simple_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:simple-small-dataset:AYWlgHow"
medium_simple_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:simple-medium-dataset:AYWlf8C2"
large_simple_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:simple-large-dataset:AYWqFlro"

large_2_epochs_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:large-dataset-2-epochs:AYWvOBRa"

large_2_steps_dataset_model_ckpt_860 = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:two-step-large-dataset-4-epochs:AYbrCyE1:ckpt-step-860"

large_dataset_gpt_4o_model = "ft:gpt-4o-2024-08-06:researcher-access-university-of-copenhagen:4o-large-full-syntax:AZDmSQ2I"
large_dataset_gpt_4o_model_ckpt_1046 = "ft:gpt-4o-2024-08-06:researcher-access-university-of-copenhagen:4o-large-full-syntax:AZDmSSPQ:ckpt-step-1046"
medium_dataset_gpt_4o_model = "ft:gpt-4o-2024-08-06:researcher-access-university-of-copenhagen:4o-medium-full-syntax:AZIz4ocK"
small_dataset_gpt_4o_model = "ft:gpt-4o-2024-08-06:researcher-access-university-of-copenhagen:4o-small-full-syntax2:AZJ88LZH"

In [34]:
model_used = small_dataset_model

In [35]:
# Create the system_prompt
example_simple = """User input: "חזון ישעיהו בןאמוץ אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"
Expected output: ["חזון ישעיהו בןאמוץ","אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"]"""

example_complex = """User input: "יביא יהוה עליך ועלעמך ועלבית אביך ימים אשר לאבאו למיום סוראפרים מעל יהודה את מלך אשור פ"
Expected output: ["יביא יהוה עליך ועלעמך ועלבית אביך ימים את מלך אשור פ" ,"אשר לאבאו למיום" ,"סוראפרים מעל יהודה"]"""


SYSTEM_PROMPT = f"Identify clauses in this Biblical Hebrew verse and return a JSON list containing the clauses, as shown in the following examples. Do not write anything else than the JSON list in your output. Example 1 {example_simple} Example 2 {example_complex}"

In [36]:
# Functions

parsed_data = []

def parse_verse(verse):
    """Send a request to the chosen OpenAI model to parse a Biblical Hebrew verse into clauses."""
    print("Parsing verse:", verse)
    for _ in range(10):
        try:
            response = client.chat.completions.create(
                model=model_used,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", 
                     "content": f"{verse}"}
                ],
                temperature=0,
            )
            # Extract and return the response
            result = response.choices[0].message.content
            parsed_data.append((verse, result))
            return verse, result
        except Exception as e:
            print(f"Error processing verse: {verse}")
            print(e)
            sleep(5)
            print("Failed to call OpenAI. Retrying.")


def process_file(input_file, output_file):
    """Read a JSONL file containing verses, process them and save results to another JSONL file."""
    global parsed_data
    parsed_data = []
    verses = []
    with open(input_file, "r", encoding="utf-8") as infile:
        for line in infile:
            data = json.loads(line)
            verses.append(data["messages"][1]["content"])
    pool = ThreadPool(processes=5)
    results = pool.map(parse_verse, verses)
    
    with open(output_file, "w", encoding="utf_8") as outfile:
        for verse, result in results:
            if result:
                output = {"verse": verse, "parsed_clauses": result}
                outfile.write(json.dumps(output, ensure_ascii=False) + "\n")

In [37]:
# Input and output files
input_jsonl_file = "data/fine_tuning_datasets/trial_1/validation_small.jsonl"
output_jsonl_file = "data/fine_tuning_datasets/trial_1/model_outputs/output_validation_small_temp_0.jsonl"

In [38]:
print(model_used)

ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-small-dataset:AYDdDFb6


In [39]:
%%time
process_file(input_jsonl_file, output_jsonl_file)

Parsing verse: Parse this verse: ויאמר שמואל אלשאול אתי שלח יהוה למשחך למלך עלעמו עלישראל ועתה שמע לקול דברי יהוה ס
Parsing verse: Parse this verse: ויאמר אלהם הוא אשר דבר יהוה שבתון שבתקדש ליהוה מחר את אשרתאפו אפו ואת אשרתבשלו בשלו ואת כלהעדף הניחו לכם למשמרת עדהבקר
Parsing verse: Parse this verse: וירד שמשון תמנתה וירא אשה בתמנתה מבנות פלשתים
Parsing verse: Parse this verse: היום הזה אחל תת פחדך ויראתך עלפני העמים תחת כלהשמים אשר ישמעון שמעך ורגזו וחלו מפניך
Parsing verse: Parse this verse: ולאכגמל עליו השיב יחזקיהו כי גבה לבו ויהי עליו קצף ועליהודה וירושלם
Parsing verse: Parse this verse: וישמע משה אתהעם בכה למשפחתיו איש לפתח אהלו ויחראף יהוה מאד ובעיני משה רע
Parsing verse: Parse this verse: וגם ללוים איןלשאת אתהמשכן ואתכלכליו לעבדתו
Parsing verse: Parse this verse: כי יהיה נערה בתולה מארשה לאיש ומצאה איש בעיר ושכב עמה
Parsing verse: Parse this verse: כי אםבמקום אשריבחר יהוה באחד שבטיך שם תעלה עלתיך ושם תעשה כל אשר אנכי מצוך
Parsing verse: Parse this verse: ועתה הנה נתן יהוה רוח שק